In [ ]:
!pip install flirpy
!pip install exifread
!pip install numpy
!pip install flirimageextractor


In [ ]:
import os
import csv
import exifread
from flirimageextractor import FlirImageExtractor


def extract_exif(image_path):
    """Extract timestamp + serial number from EXIF."""
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    return {
        "timestamp": str(tags.get("EXIF DateTimeOriginal", "")),
        "camera_sn": str(tags.get("Image SerialNumber", "")),
    }


def extract_temperatures(image_path):
    fie = FlirImageExtractor()
    fie.process_image(image_path)

    thermal_np = fie.get_thermal_np()

    if thermal_np is None:
        return None, None, None

    return float(thermal_np.min()), float(thermal_np.max()), float(thermal_np.mean())


# ------------------ MAIN SCRIPT ------------------

folder = r"C:\Users\st1mohamna\OneDrive - Prometeon\Desktop\IgniteSync\100_FLIR"
output_csv = os.path.join(folder, "flir_data.csv")

rows = []

for file in os.listdir(folder):
    if file.lower().endswith((".jpg", ".jpeg")):

        image_path = os.path.join(folder, file)

        # --- EXIF ---
        exif = extract_exif(image_path)

        # --- THERMAL TEMPERATURES ---
        tmin, tmax, tavg = extract_temperatures(image_path)

        rows.append({
            "image_name": file,
            "timestamp": exif["timestamp"],
            "camera_sn": exif["camera_sn"],
            "temp_min": tmin,
            "temp_max": tmax,
            "temp_avg": tavg,
        })


# --- Write CSV ---
with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys())
    writer.writeheader()
    writer.writerows(rows)

print("✅ CSV created at:", output_csv)


File paths successfully generated:
DIRP: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.7\windows/release_x64/libdirp.dll
DIRP Sub: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.7\windows/release_x64/libv_dirp.dll
IIRP: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.7\windows/release_x64/libv_iirp.dll
ExifTool: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.7\exiftool-12.35.exe
File paths successfully generated:
DIRP: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.7\windows/release_x64/libdirp.dll
DIRP Sub: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.7\windows/release_x64/libv_dirp.dll
IIRP: c:\Users\st1mohamna\AppData\Local\anaconda3\Lib\site-packages\dji_executables\dji_thermal_sdk_v1.

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\st1mohamna\\OneDrive - Prometeon\\Desktop\\IgniteSync\\100_FLIR\\flir_data.csv'

In [ ]:
import os
import exifread
from flirimageextractor import FlirImageExtractor
import pandas as pd

def extract_exif(image_path):
    """Extract timestamp + serial number from EXIF."""
    with open(image_path, 'rb') as f:
        tags = exifread.process_file(f)

    return {
        "timestamp": str(tags.get("EXIF DateTimeOriginal", "")),
        "camera_sn": str(tags.get("Image SerialNumber", "")),
    }

def extract_temperatures(image_path):
    """Extract thermal matrix and compute min/max/avg."""
    fie = FlirImageExtractor()
    fie.process_image(image_path)

    thermal_np = fie.get_thermal_np()

    if thermal_np is None:
        return None, None, None

    return float(thermal_np.min()), float(thermal_np.max()), float(thermal_np.mean())

# ------------------ MAIN SCRIPT ------------------

folder = r"C:\Users\st1mohamna\OneDrive - Prometeon\Desktop\IgniteSync\100_FLIR"

data = []  # list of dictionaries

for file in os.listdir(folder):
    if file.lower().endswith((".jpg", ".jpeg")):
        image_path = os.path.join(folder, file)

        # --- EXIF ---
        exif = extract_exif(image_path)

        # --- THERMAL TEMPERATURES ---
        tmin, tmax, tavg = extract_temperatures(image_path)

        data.append({
            "image_name": file,
            "timestamp": exif["timestamp"],
            "camera_sn": exif["camera_sn"],
            "temp_min": tmin,
            "temp_max": tmax,
            "temp_avg": tavg,
        })

# Convert to DataFrame
df = pd.DataFrame(data)

# Optional: save to CSV if needed
output_csv = os.path.join(folder, "flir_data.csv")
df.to_csv(output_csv, index=False, encoding="utf-8")

print("✅ DataFrame created. CSV saved at:", output_csv)
df.head()


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [10]:
################################################## MAZEN ##################################################

import os
import csv
import json
import subprocess
import numpy as np
import contextlib
import sys
from flirimageextractor import FlirImageExtractor

# --- CONFIGURATION ---

# 1. SMART PATH FINDER (Works in both .py and .ipynb)
try:
    # Try getting the path using __file__ (works for .py scripts)
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # If that fails (Jupyter Notebooks), just use the current working directory
    base_dir = os.getcwd()

# 2. Define folders
input_folder = os.path.join(base_dir, "100_FLIR")
output_csv = os.path.join(base_dir, "Batch_Thermal_Report.csv")

# 3. ExifTool Path
exiftool_filename = "exiftool-12.35.exe"
exiftool_path = os.path.join(base_dir, exiftool_filename)

# --- HELPER FUNCTIONS ---

def get_metadata_with_exiftool(file_path):
    if not os.path.exists(exiftool_path):
        tool_cmd = "exiftool"
    else:
        tool_cmd = exiftool_path

    try:
        cmd = [
            tool_cmd,
            "-DateTimeOriginal",
            "-CameraModel",
            "-CameraSerialNumber",
            "-Emissivity",
            "-ObjectDistance",
            "-j",
            file_path
        ]
        
        creation_flags = 0
        if os.name == 'nt': 
            creation_flags = subprocess.CREATE_NO_WINDOW
            
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=creation_flags)
        
        if result.stdout:
            return json.loads(result.stdout)[0]
    except Exception:
        pass
    return {}

def get_processed_files(csv_path):
    processed = set()
    if not os.path.exists(csv_path):
        return processed
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader, None)
            for row in reader:
                if row:
                    processed.add(row[0])
    except Exception:
        pass 
    return processed

def process_folder_incremental():
    headers = [
        "File Name", "Timestamp", "Camera Model", "Serial Number", 
        "Center Temp (°C)", "Max Temp (°C)", "Min Temp (°C)", 
        "Avg Temp (°C)", "Delta Temp (°C)", 
        "Emissivity", "Distance", "Status"
    ]
    
    # Validation Checks
    if not os.path.exists(input_folder):
        print(f"Error: Input folder not found at:\n{input_folder}")
        print(f"(Looking in base dir: {base_dir})")
        return

    # 1. IDENTIFY WORK
    all_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".jpg")]
    processed_files = get_processed_files(output_csv)
    files_to_process = [f for f in all_files if f not in processed_files]
    
    print(f"Total Images: {len(all_files)}")
    print(f"Already Done: {len(processed_files)}")
    print(f"New to Process: {len(files_to_process)}")
    
    if not files_to_process:
        print("Nothing new to add. Exiting.")
        return

    # 2. INITIALIZE FLIR TOOL (SILENTLY)
    print("Initializing FLIR Extractor...", end="\r")
    try:
        with contextlib.redirect_stdout(None):
            flir = FlirImageExtractor()
    except Exception as e:
        print(f"\nError initializing FLIR extractor: {e}")
        return

    # 3. PREPARE CSV (Handle Permission Errors)
    file_exists = os.path.exists(output_csv)
    
    try:
        f = open(output_csv, mode='a', newline='', encoding='utf-8')
    except PermissionError:
        print(f"\n\nERROR: Permission Denied for '{output_csv}'")
        print(">>> PLEASE CLOSE THE CSV FILE IN EXCEL AND TRY AGAIN. <<<")
        return
    except Exception as e:
        print(f"\nError opening CSV: {e}")
        return

    # Use 'with' to ensure file closes even if script crashes
    with f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(headers)
        
        # 4. PROCESS LOOP
        for i, filename in enumerate(files_to_process):
            full_path = os.path.join(input_folder, filename)
            
            # Progress Bar
            print(f"Processing [{i+1}/{len(files_to_process)}]: {filename}" + " "*20, end="\r")
            
            try:
                # --- A. Thermal Data ---
                with contextlib.redirect_stdout(None):
                    flir.process_image(full_path)
                    
                thermal_data = flir.get_thermal_np()
                
                min_t = np.min(thermal_data)
                max_t = np.max(thermal_data)
                avg_t = np.mean(thermal_data)
                delta_t = max_t - min_t
                h, w = thermal_data.shape
                center_t = thermal_data[h//2, w//2]
                
                # --- B. Metadata ---
                meta = get_metadata_with_exiftool(full_path)
                
                serial = meta.get("CameraSerialNumber", "Unknown")
                timestamp = meta.get("DateTimeOriginal", "Unknown")
                camera_model = meta.get("CameraModel", "Unknown")
                emissivity = meta.get("Emissivity", "N/A")
                distance = meta.get("ObjectDistance", "N/A")
                
                # --- C. Write Row ---
                writer.writerow([
                    filename, timestamp, camera_model, serial, 
                    f"{center_t:.1f}", f"{max_t:.1f}", f"{min_t:.1f}", 
                    f"{avg_t:.1f}", f"{delta_t:.1f}", 
                    emissivity, distance, "Success"
                ])
                
                # Force save immediately so data isn't lost if script crashes
                f.flush()
                
            except Exception as e:
                writer.writerow([filename, "", "", "", "", "", "", "", "", "", "", f"Error: {str(e)}"])

    print(f"\n\nDone! Added {len(files_to_process)} rows to: {output_csv}")

if __name__ == "__main__":
    try:
        process_folder_incremental()
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

Total Images: 11
Already Done: 0
New to Process: 11
Processing [11/11]: FLIR0013.jpg                    

Done! Added 11 rows to: c:\flir e5 pro\Flir-e5-pro-project\image extraction\flir ignite sync\Batch_Thermal_Report.csv


In [7]:
################################################## MAZEN  - Timestamp Checker ##################################################

import os
import csv
import json
import subprocess
import numpy as np
import contextlib
from flirimageextractor import FlirImageExtractor

# --- CONFIGURATION ---

# 1. SMART PATH FINDER (Works in both .py and .ipynb)
try:
    # Try getting the path using __file__ (works for .py scripts)
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # If that fails (Jupyter Notebooks), just use the current working directory
    base_dir = os.getcwd()

# 2. Define folders
input_folder = os.path.join(base_dir, "100_FLIR")
output_csv = os.path.join(base_dir, "Batch_Thermal_Report.csv")

# 3. ExifTool Path
exiftool_filename = "exiftool-12.35.exe"
exiftool_path = os.path.join(base_dir, exiftool_filename)

# --- HELPER FUNCTIONS ---

def get_metadata_with_exiftool(file_path):
    if not os.path.exists(exiftool_path):
        tool_cmd = "exiftool"
    else:
        tool_cmd = exiftool_path

    try:
        cmd = [
            tool_cmd,
            "-DateTimeOriginal",
            "-CameraModel",
            "-CameraSerialNumber",
            "-Emissivity",
            "-ObjectDistance",
            "-j",
            file_path
        ]
        
        creation_flags = 0
        if os.name == 'nt': 
            creation_flags = subprocess.CREATE_NO_WINDOW
            
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=creation_flags)
        
        if result.stdout:
            return json.loads(result.stdout)[0]
    except Exception:
        pass
    return {}

def get_processed_timestamps(csv_path):
    """Reads the CSV and returns a set of Timestamps (Column 1)"""
    processed = set()
    if not os.path.exists(csv_path):
        return processed
    try:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader, None) # Skip header
            for row in reader:
                # Ensure the row has enough columns and Timestamp (index 1) exists
                if len(row) > 1 and row[1]: 
                    processed.add(row[1])
    except Exception:
        pass 
    return processed

def process_folder_incremental():
    headers = [
        "File Name", "Timestamp", "Camera Model", "Serial Number", 
        "Center Temp (°C)", "Max Temp (°C)", "Min Temp (°C)", 
        "Avg Temp (°C)", "Delta Temp (°C)", 
        "Emissivity", "Distance", "Status"
    ]
    
    # Validation Checks
    if not os.path.exists(input_folder):
        print(f"Error: Input folder not found at:\n{input_folder}")
        print(f"(Looking in base dir: {base_dir})")
        return

    # 1. IDENTIFY WORK
    # We load all JPGs, but we don't know which are "new" until we read their metadata inside the loop
    all_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".jpg")]
    
    # Load Timestamps instead of Filenames
    processed_timestamps = get_processed_timestamps(output_csv)
    
    print(f"Total Images found in folder: {len(all_files)}")
    print(f"Timestamps already in CSV: {len(processed_timestamps)}")
    print("Scanning metadata to identify new images...")
    
    # 2. INITIALIZE FLIR TOOL (SILENTLY)
    print("Initializing FLIR Extractor...", end="\r")
    try:
        with contextlib.redirect_stdout(None):
            flir = FlirImageExtractor()
    except Exception as e:
        print(f"\nError initializing FLIR extractor: {e}")
        return

    # 3. PREPARE CSV (Handle Permission Errors)
    file_exists = os.path.exists(output_csv)
    
    try:
        f = open(output_csv, mode='a', newline='', encoding='utf-8')
    except PermissionError:
        print(f"\n\nERROR: Permission Denied for '{output_csv}'")
        print(">>> PLEASE CLOSE THE CSV FILE IN EXCEL AND TRY AGAIN. <<<")
        return
    except Exception as e:
        print(f"\nError opening CSV: {e}")
        return

    # Use 'with' to ensure file closes even if script crashes
    with f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(headers)
        
        # 4. PROCESS LOOP
        # We loop through ALL files, extract metadata, then decide whether to skip
        processed_count = 0
        skipped_count = 0

        for i, filename in enumerate(all_files):
            full_path = os.path.join(input_folder, filename)
            
            # Progress Bar (Simplified as we scan everyone)
            print(f"Checking [{i+1}/{len(all_files)}]: {filename}" + " "*20, end="\r")
            
            try:
                # --- A. Metadata FIRST (To check Timestamp) ---
                meta = get_metadata_with_exiftool(full_path)
                
                # Get the timestamp
                current_timestamp = meta.get("DateTimeOriginal", "Unknown")

                # CHECK: Compare Timestamp with CSV records
                if current_timestamp in processed_timestamps and current_timestamp != "Unknown":
                    # If found, skip heavy processing
                    skipped_count += 1
                    continue

                # --- B. Thermal Data (Only if Timestamp is new) ---
                with contextlib.redirect_stdout(None):
                    flir.process_image(full_path)
                    
                thermal_data = flir.get_thermal_np()
                
                min_t = np.min(thermal_data)
                max_t = np.max(thermal_data)
                avg_t = np.mean(thermal_data)
                delta_t = max_t - min_t
                h, w = thermal_data.shape
                center_t = thermal_data[h//2, w//2]
                
                serial = meta.get("CameraSerialNumber", "Unknown")
                camera_model = meta.get("CameraModel", "Unknown")
                emissivity = meta.get("Emissivity", "N/A")
                distance = meta.get("ObjectDistance", "N/A")
                
                # --- C. Write Row ---
                writer.writerow([
                    filename, current_timestamp, camera_model, serial, 
                    f"{center_t:.1f}", f"{max_t:.1f}", f"{min_t:.1f}", 
                    f"{avg_t:.1f}", f"{delta_t:.1f}", 
                    emissivity, distance, "Success"
                ])
                
                # Force save immediately
                f.flush()
                processed_count += 1
                
                # Add this new timestamp to our local set so duplicates in the same run are skipped
                processed_timestamps.add(current_timestamp)
                
            except Exception as e:
                # If an error occurs, we still log it, but we can't rely on timestamp check for errors
                writer.writerow([filename, "", "", "", "", "", "", "", "", "", "", f"Error: {str(e)}"])

    print(f"\n\nDone!")
    print(f"Skipped (Already Exists): {skipped_count}")
    print(f"Processed (New): {processed_count}")

if __name__ == "__main__":
    try:
        process_folder_incremental()
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

Total Images found in folder: 11
Timestamps already in CSV: 11
Scanning metadata to identify new images...
Checking [11/11]: FLIR0013.jpg                    

Done!
Skipped (Already Exists): 11
Processed (New): 0


In [ ]:
################################################### MAZEN  - DataFrame Viewer ##################################################
import pandas as pd

df = pd.read_csv(output_csv)
df

,File Name,Timestamp,Camera Model,Serial Number,Center Temp (°C),Max Temp (°C),Min Temp (°C),Avg Temp (°C),Delta Temp (°C),Emissivity,Distance,Status
0,FLIR0003.jpg,2025:11:24 14:30:18.107+02:00,FLIR E5 Pro,13333134,23.0,23.9,21.7,23.1,2.2,0.95,1.00 m,Success
1,FLIR0004.jpg,2025:11:24 14:31:19.177+02:00,FLIR E5 Pro,13333134,28.5,54.5,21.3,28.4,33.2,0.95,1.00 m,Success
2,FLIR0005.jpg,2025:11:24 14:40:36.898+02:00,FLIR E5 Pro,13333134,22.4,23.0,22.2,22.6,0.8,0.95,1.00 m,Success
3,FLIR0006.jpg,2025:11:24 15:19:36.703+02:00,FLIR E5 Pro,13333134,30.0,61.0,21.3,27.4,39.7,0.95,1.00 m,Success
4,FLIR0007.jpg,2025:11:25 09:59:39.012+02:00,FLIR E5 Pro,13333134,26.1,31.4,23.7,24.8,7.7,0.95,1.00 m,Success
5,FLIR0008.jpg,2025:11:25 10:25:57.424+02:00,FLIR E5 Pro,13333134,27.6,37.0,23.2,27.0,13.7,0.95,1.00 m,Success
6,FLIR0009.jpg,2025:11:25 10:26:10.676+02:00,FLIR E5 Pro,13333134,25.8,35.9,24.0,26.5,11.8,0.95,1.00 m,Success
7,FLIR0010.jpg,2025:11:25 14:19:38.198+02:00,FLIR E5 Pro,13333134,25.4,31.8,19.6,25.0,12.1,0.95,1.00 m,Success
8,FLIR0011.jpg,2025:11:25 14:21:46.913+02:00,FLIR E5 Pro,13333134,35.6,36.3,21.9,29.0,14.4,0.95,1.00 m,Success
9,FLIR0012.jpg,2025:11:26 09:47:52.605+02:00,FLIR E5 Pro,13333134,17.1,30.7,12.5,20.5,18.2,0.95,1.00 m,Success
